In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
import re


2024-06-06 15:09:29.493693: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-06 15:09:29.493911: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-06 15:09:29.496591: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-06 15:09:29.528884: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 15:09:30.154724: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [2]:
df = pd.read_csv("Train (2).csv")
df.sample(10)

,Tweet_ID,tweet,type
26344,ID_O0UA4JZY,Sure didn’t the himself write a character refe...,sexual_violence
34489,ID_VF2WPCJX,Same respect he showed that lady he raped ???...,sexual_violence
25460,ID_N8V0EMRY,Lmao you all are so funny ... like Laycon rape...,sexual_violence
24695,ID_MIE6RMS7,Him my consent at alll. So im crying bc im get...,sexual_violence
32744,ID_TTH1S79F,I was called in by uni bigwigs as a male stude...,sexual_violence
33700,ID_UOAXH3EZ,"Or is it the phone calls of a friend, Doki: ...",sexual_violence
10941,ID_9WP8URJM,So it would stink up later. Kept following her...,sexual_violence
27162,ID_OQYQ97P4,Ya lol George was just gunna leave me on lot l...,sexual_violence
5354,ID_4SLMHQBC,My auntie threw holy water on my rapist on the...,sexual_violence
30086,ID_RDJ5LUUJ,"oh sorry no, her family member was a pedofile....",sexual_violence


In [3]:
len(df)

39650

In [4]:
df.isna().sum()

Tweet_ID    0
tweet       0
type        0
dtype: int64

In [5]:
df.duplicated().sum()

0

In [6]:
df.nunique()

Tweet_ID    39650
tweet       39642
type            5
dtype: int64

In [7]:
#separation des données en features et labels 
X = df["tweet"].values
y = df["type"].values

In [8]:
#tokenization du texts 
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X)
sequences =tokenizer.texts_to_sequences(X)

In [21]:
tokenizer.word_docs

defaultdict(int,
            {'by': 5233,
             'work': 617,
             'guy': 2475,
             'tryna': 55,
             'last': 810,
             'just': 5177,
             'too': 1974,
             'once': 811,
             'doing': 670,
             'at': 5266,
             'me': 38251,
             'night': 965,
             'smoke': 52,
             'and': 22347,
             'but': 7989,
             'that': 13889,
             'even': 2707,
             'was': 15569,
             'my': 17249,
             'had': 4631,
             'house': 889,
             'with': 7981,
             'like': 5529,
             'smoked': 16,
             'got': 3409,
             'wasn’t': 596,
             'wanted': 1015,
             'raped': 31961,
             'to': 22871,
             'it': 10979,
             'for': 10040,
             'dream': 123,
             'a': 21040,
             'actually': 888,
             'be': 5717,
             'he': 32183,
             'much': 1113

In [9]:
#padding 
maxlen = 100
padded_sequences = pad_sequences(sequences,maxlen=maxlen)

In [10]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.2, random_state=42)


In [11]:
# Convertir les étiquettes en nombres
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test= label_encoder.transform(y_test)

In [12]:
#concstruiction du modele rnn
# Construction du modèle RNN
model = Sequential()
model.add(Embedding(10000, 64, input_length=maxlen))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

/home/sonya/Documents/projet_perso/tweet_classification_NLP/.venv/lib/python3.9/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2024-06-06 15:09:32.735752: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-06 15:09:32.736068: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [13]:
# Compilation du modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# Définition des callbacks pour l'arrêt anticipé
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [15]:
# Entraînement du modèle
history = model.fit(X_train, y_train, epochs=20, batch_size=15, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/20
1692/1692 ━━━━━━━━━━━━━━━━━━━━ 31s 18ms/step - accuracy: 0.1485 - loss: -92.5331 - val_accuracy: 0.1504 - val_loss: -301.6323
Epoch 2/20
1692/1692 ━━━━━━━━━━━━━━━━━━━━ 31s 18ms/step - accuracy: 0.1501 - loss: -369.6904 - val_accuracy: 0.1504 - val_loss: -571.1514
Epoch 3/20
1692/1692 ━━━━━━━━━━━━━━━━━━━━ 31s 18ms/step - accuracy: 0.1512 - loss: -638.4811 - val_accuracy: 0.1504 - val_loss: -840.3177
Epoch 4/20
1692/1692 ━━━━━━━━━━━━━━━━━━━━ 31s 18ms/step - accuracy: 0.1462 - loss: -913.6166 - val_accuracy: 0.1504 - val_loss: -1109.4000
Epoch 5/20
1692/1692 ━━━━━━━━━━━━━━━━━━━━ 32s 19ms/step - accuracy: 0.1500 - loss: -1177.5540 - val_accuracy: 0.1504 - val_loss: -1378.5201
Epoch 6/20
1692/1692 ━━━━━━━━━━━━━━━━━━━━ 34s 20ms/step - accuracy: 0.1491 - loss: -1449.6241 - val_accuracy: 0.1504 - val_loss: -1647.5509
Epoch 7/20
1692/1692 ━━━━━━━━━━━━━━━━━━━━ 36s 21ms/step - accuracy: 0.1507 - loss: -1716.7100 - val_accuracy: 0.1504 - val_loss: -1916.8717
Epoch 8/20
1692/1692 ━━━━━━━